In [1]:
!pip install gnss_lib_py|

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [2]:
import numpy as np
import gnss_lib_py as glp
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import pandas as pd

https://drive.google.com/drive/folders/17vNrS_-Kq0msv2r9VHDkry3aEAxUcdAS

### Pseudorange computation

In [5]:
def compute_pseudoranges(raw_data):
    nanosecs_in_week = 604800e9
    speed_of_light = 299792458
    week_no = np.floor(-raw_data['FullBiasNanos']/nanosecs_in_week)
    corrected_rx_time = raw_data['TimeNanos'] + raw_data['TimeOffsetNanos']
    adjusted_rx_bias = raw_data['FullBiasNanos'] + raw_data['BiasNanos']
    rx_time_gps_frame = corrected_rx_time - adjusted_rx_bias
    rx_time_week_frame = rx_time_gps_frame - week_no * nanosecs_in_week
    rho_nanosecs = rx_time_week_frame - raw_data['ReceivedSvTimeNanos']
    rho_m = rho_nanosecs * speed_of_light/1e9
    return rho_m

In [198]:
rover_path = './Oval/RoundLoop1_gnss_log_2024_11_17_16_52_31.txt'
base_path = './Oval/RoundLoop1_BaseStation_gnss_log_2024_11_17_16_52_30.txt'

In [199]:
rover_data = glp.AndroidRawGnss(input_path=rover_path,
                              filter_measurements=True,
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

base_data = glp.AndroidRawGnss(input_path=base_path,
                              filter_measurements=True,
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

rover_rho = compute_pseudoranges(rover_data)
base_rho = compute_pseudoranges(base_data)

sv_time_uncertainty removed 1
sv_time_uncertainty removed 1


### Pull the ephemeris (don't work)

In [115]:
rover_gps_millis_start, rover_gps_millis_end = rover_data['gps_millis'][0], rover_data['gps_millis'][-1] 
base_gps_millis_start, base_gps_millis_end = base_data['gps_millis'][0], base_data['gps_millis'][-1]

In [116]:
rover_ephemeris_file = glp.load_ephemeris(file_type="sp3",
                              gps_millis=np.array([rover_gps_millis_start, rover_gps_millis_start]),
                              verbose=True)

base_ephemeris_file = glp.load_ephemeris(file_type="sp3",
                              gps_millis=np.array([base_gps_millis_start, base_gps_millis_start]),
                              verbose=True)

ephemeris dates needed: [datetime.date(2024, 11, 17), datetime.date(2024, 11, 18)]
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/sp3/COD0OPSRAP_20243220000_01D_05M_ORB.SP3
FTP downloading /gnss/products/2341/COD0OPSRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
550 Failed to open file. Failed to retrieve /gnss/products/2341/COD0OPSRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/sp3/COD0OPSRAP_20243220000_01D_05M_ORB.SP3
FTP downloading /gnss/products/2341/COD0OPSRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/sp3/COD0OPSRAP_20243220000_01D_05M_ORB.SP3
FTP downloading /gnss/products/2341/COD0OPSRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov


error_perm: 550 Failed to open file. Failed to retrieve /gnss/products/2341/COD0OPSRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov

### Calculate true ranges

In [200]:
BASE_STATION_LOCATION_1 = np.array([37.429711,-122.169381,25]).reshape(3, 1)
BASE_STATION_LOCATION_2 = np.array([37.4290631,-122.1723136,25]).reshape(3, 1) # multipath test

DURAND = np.array([[37.426906], [-122.173278], [21.010]]) 

In [143]:
## Load the RinexNav data for that day
rinex_nav_paths = glp.load_ephemeris(
    "rinex_nav",
    np.array(np.min(base_data['gps_millis']),np.max(base_data['gps_millis'])),
    verbose=True,
)
rinex_nav = glp.RinexNav(rinex_nav_paths, verbose=True)

ephemeris dates needed: [datetime.date(2024, 11, 17), datetime.date(2024, 11, 18)]
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/rinex/nav/BRDC00WRD_S_20243220000_01D_MN.rnx
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/rinex/nav/BRDC00WRD_S_20243230000_01D_MN.rnx
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/rinex/nav/BRDC00WRD_S_20243220000_01D_MN.rnx
using previously downloaded file:
 /home/gigavoltflash/stanford_classes/AA_272/dgnss_aa272/data/ephemeris/rinex/nav/BRDC00WRD_S_20243230000_01D_MN.rnx


/home/gigavoltflash/miniconda3/lib/python3.9/site-packages/gnss_lib_py/parsers/rinex_nav.py:295: RuntimeWarning: No ionospheric parameters found in RINEX file
  warnings.warn("No ionospheric parameters found in RINEX file",
/home/gigavoltflash/miniconda3/lib/python3.9/site-packages/gnss_lib_py/parsers/rinex_nav.py:295: RuntimeWarning: No ionospheric parameters found in RINEX file
  warnings.warn("No ionospheric parameters found in RINEX file",


In [ ]:
def find_sv_pos(rinex, sv_id, gnss_id, gps_millis):
    """ finding sv position with ephemeris data FOR DURAND receiver"""
    n = len(sv_id)
    position_info = np.zeros((n, 3))
    for i in range(n):
        sv_at_millis = glp.find_sv_states(gps_millis[i], rinex)
        cropped_sv_at_millis = sv_at_millis.where('sv_id', sv_id[i], 'eq')
        gnss_id_at_millis = cropped_sv_at_millis.where('gnss_id', gnss_id[i], 'eq')
        position_info[i, 0] = np.mean(gnss_id_at_millis['x_sv_m'])
        position_info[i, 1] = np.mean(gnss_id_at_millis['y_sv_m'])
        position_info[i, 2] = np.mean(gnss_id_at_millis['z_sv_m'])
    return position_info

# FOR DURAND DATA
# def get_measured_pseudorange(df,sv_id):
#     """ get measured pseudorange of specific sv for durand receiver"""
#     filtered_df = df[df.iloc[:,4] == str(sv_id)]
#     return filtered_df.iloc[:,14]

def distance_to_groundtruth(sv_pos, ground_truth):
    """ calculate ground truth distance from SV to Durand receiver"""
    gt_ecef = glp.geodetic_to_ecef(ground_truth)
    distance = np.linalg.norm(sv_pos[:, :, None] - gt_ecef[None, :, :], axis=1)
    return distance

def correction(true_range, base_pseudorange, rover_pseudorange):
    """ true_range: ground truth distance from distance_to_groundtruth function
    base_pseudorange: pseudorange for base station
    pseudorange: pseudorange for phone """
    e = true_range - base_pseudorange
    adjusted_pseudorange = rover_pseudorange + e
    return adjusted_pseudorange

# def get_pseudorange(raw_data, sv_id):
#     """ get measure pseudorange of the rover with specified sv_id"""
#     cropped_data = raw_data.where("sv_id", sv_id, "eq")
#     pseudorange = compute_pseudoranges(cropped_data)
#     return pseudorange

In [171]:

sv_pos = find_sv_pos(rinex_nav, base_data['sv_id'], base_data['gnss_id'], base_data['gps_millis'])

# sv_pos = find_sv_pos(rover_ephemeris, 6)
# true_ranges = distance_to_groundtruth()

### Actual dGNSS shit

In [202]:
base_pseudorange = compute_pseudoranges(base_data)
true_range = distance_to_groundtruth(sv_pos, BASE_STATION_LOCATION_1).flatten()
rover_pseudorange = compute_pseudoranges(rover_data)
corrected_pseudorange = correction(true_range[:200], base_pseudorange[:200], rover_pseudorange[:200])

In [206]:
corrected_pseudorange

array([2.03984206e+14, 2.03984208e+14, 2.03984208e+14, 2.03984205e+14,
       2.03984206e+14, 2.03984206e+14, 2.03984209e+14, 2.03984206e+14,
       2.03984208e+14, 1.81314503e+14,            nan,            nan,
                  nan, 1.81314481e+14,            nan, 1.81314500e+14,
                  nan, 1.81314499e+14,            nan, 1.81314504e+14,
                  nan, 1.81314504e+14, 1.81314505e+14, 1.81314503e+14,
                  nan, 1.81314504e+14, 1.81314506e+14, 2.03984206e+14,
       2.03984208e+14, 2.03984208e+14, 2.03984205e+14, 2.03984206e+14,
       2.03984205e+14, 2.03984209e+14, 2.03984186e+14, 2.03984208e+14,
       1.81314503e+14, 1.81314502e+14, 1.81314499e+14, 1.81314499e+14,
       1.81314501e+14, 1.81314499e+14, 1.81314501e+14, 1.81314503e+14,
       1.81314499e+14,            nan,            nan,            nan,
                  nan, 1.81314504e+14, 1.81314482e+14,            nan,
       1.81314504e+14,            nan, 2.03984205e+14,            nan,
      